In [13]:
setwd(paste0(Sys.getenv('R_SOURCES'), '/house_prices'))
source('main.R')
house_prices$helpers$import_libs()

combined_dataset <-
    house_prices$helpers$load_data() %>%
    (house_prices$outliers$remove_outliers) %>%
    (house_prices$missing$fix_all) %>%
    mutate(
        price_log = log(SalePrice)
    ) %>%
    select(-SalePrice, -Id)

combined_dataset %>% dim

test_that("should be no NA values except SalePrice column", {
    expect_equal(sum(is.na(combined_dataset %>% select(-price_log))), 0)
})

[1] 2917   81

In [15]:
calc_quantiles <- function(sample, probs = NULL) {
    if (is.null(probs)) {
        probs <- c(.25, .5, .75)
    }
    quantile(sample, probs, na.rm=TRUE)
}

calc_quantiles(combined_dataset[['price_log']])

25%      50%      75% 
11.77471 12.00151 12.27373

In [46]:
#calc_rating_for_sample

calc_rating_for_sample <- function(sample, quantiles) {
    sample <- sample[!is.na(sample)]
    cdf = ecdf(sample)
    cdf_points <- c(0, cdf(quantiles), 1)
    probs <- tail(cdf_points, -1) - head(cdf_points, -1)
    rating <- sum(probs * c(1:length(probs)))
    rating
}

calc_rating_for_sample(
    c(1, 2, 3, 4, 5, 6, 7, 8), 
    c(2.5, 4.5, 6.5))

[1] 2.5

In [11]:
calc_rating_for_groups <- function(df) {
    df <- 
        df %>%
        select(Alley, BldgType, price_log) %>%
        filter(!is.na(price_log))

    global_quantiles <- calc_global_quantiles(df)

    df %>%
    gather(var, value, -price_log) %>%
    group_by(var, value) %>%
    nest %>%
    mutate(
       rating = map_dbl(data, function (df) {
           calc_rating_for_sample(df[['price_log']], global_quantiles)
       })
    ) %>%
    select(-data) %>%
    rbind(list(NA, NA, 0.25*(1+2+3+4)))
}

calc_rating_for_groups(combined_dataset)

var,value,rating
Alley,_none_,2.531822
Alley,Grvl,1.500000
Alley,Pave,2.536585
BldgType,1Fam,2.564039
BldgType,2fmCon,1.580645
BldgType,Duplex,1.692308
BldgType,TwnhsE,2.622807
BldgType,Twnhs,1.883721
NA,NA,2.500000
